<a href="https://colab.research.google.com/github/ewilson2023/History-Name-Mashup-Generator/blob/main/HistoryChalengeNameGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this
A "fantasy" name generator. Picks syllables of names from Ancient Egypt, Ancient Greece, Ancient Rome, and Ancient Near East and generates a mashup name

## How to use it
1) if the table of contents isn't visible, click the hamburger button at the top of the left vertical menu bar.
2) Click "Main".
3) If on mobile, close the table of contents to see the code
4) At the top of the page, you'll see a menu with options for "Commands", "Code", "Text", and "Run all".
5) Click "Run all".
6) To generate again, click the play button under the "Main" label.

# Classes and Utilities

In [131]:
# class for each cultur, storing sorted dictionaries

class Culture:
  def __init__(self, p, m, s, name):
    self.name = name
    self.pre = sort(p, name, "pre")
    self.mid = sort(m, name, "mid")
    self.suf = sort(s, name, "suf")





*   dicts lack explicit structure
* TypedDict: a fixed-shape dictionary where the keys are always the same bucket names, and each key always maps to the same kind of value


   

In [132]:
from typing import TypedDict

# takes the form
# "name":   list["string"],
# "name":   list[int],
"""
SortBuckets is not a class at runtime. A SortedBuckets b
is a dictionary that has specific keys and value types.
b["all"] is a list of strings, b["c_W"] is a list of ints
"""
class SortBuckets(TypedDict):
    all: list[str]
    vv: list[int]; cc: list[int]
    vc: list[int]; cv: list[int]
    c: list[int];  v: list[int]
    c_W: list[int]; v_W: list[int]
    W_c: list[int]; W_v: list[int]

In [133]:
# return a dictionary sorted by consonant/vowel ending
def sort(list: list[str], cult: str, word_part: str) -> dict:
    # W means wildcard
    dict = {
          "all" :[],
          "vv": [], "cc": [],
          "vc": [], "cv": [],
          "c": [],  "v": [],

          "C_W": [], "V_W": [],

          "c_W": [], "v_W": [],
          "W_c": [], "W_v": [],
      }

    # check if this culture is Rome or Greece for suffix sorting
    if cult in {"Roman", "Greek"} and word_part == "suf":
      RG_suffix: bool = True
    else:
      RG_suffix: bool = False

    """
      if syllable's last two chars are consonants, or is only one consonant,
       do not allow a syllable with ' or - to follow it

      if greek or roman suffix, forbid any ' or - from end of current name,
       ensure proper consonant/vowel match
    """


    for idx, str in enumerate(list):
        dict["all"].append(str)

        str= str.lower()

        first = str[0]
        last = str[len(str) - 1]

        # if syllable is only one char
        if len(str) == 1:
          if first_is_cons(str):
            dict["c"].append(idx)
            dict["cc"].append(idx)
            dict["c_W"].append(idx)
            dict["W_c"].append(idx)
            dict["C_W"].append(idx)
          else:
            dict["v"].append(idx)
            dict["vv"].append(idx)
            dict["v_W"].append(idx)
            dict["W_v"].append(idx)
            dict["V_W"].append(idx)
          continue

        # if the syllable start with a - or ', add to both "any" lists
        elif first == "-" or first == "'":
          dict["c_W"].append(idx)
          dict["v_W"].append(idx)

        elif RG_suffix:
          if first_is_cons(str):
            dict["C_W"].append(idx)
          else:
            dict["V_W"].append(idx)

        # if syllable has 2 or more chars
        else:
          # if first char is a consonant or consonant-type y
          if first_is_cons(str):
            dict["c_W"].append(idx)
            dict["C_W"].append(idx)

            if last_is_cons(str):
              dict["cc"].append(idx)
              dict["W_c"].append(idx)
            else:
              dict["cv"].append(idx)
              dict["W_v"].append(idx)

          else: # first is a vowel
            dict["v_W"].append(idx)
            dict["V_W"].append(idx)

            if last_is_cons(str):
              dict["vc"].append(idx)
              dict["W_c"].append(idx)
            else:
              dict["vv"].append(idx)
              dict["W_v"].append(idx)


    return dict


In [134]:
def isVowel(char: str) -> bool:
  char = char.lower()
  vowels = ["a", "e", "i", "o", "u", "ë"]

  if char in vowels:
    return True
  else:
    return False

def isConsonant(char: str) -> bool:
  return not isVowel(char)

# True if a string is all consonants
def all_consonants(str: str) -> bool:

  for i, char in enumerate(str):
    # y can only be consonant-like if theres a vowel in the str
    if str[i] == "y":
      continue
    elif i > 0 and str[i] == "u" and str[i-1] == "q":
      continue
    if not isConsonant(char):
      return False
  return True

# return false is a starting y is vowel-like
def first_is_cons(str: str) -> bool:
  if len(str) == 1 and str[0] == "y":
    return False

  # any normal cons that is not 'y' is a consonant
  non_y_cons: bool = isConsonant(str[0]) and str[0] != "y"

  # a starting 'y' is a cons if any normal vowel follows
  y_cons: bool = str[0] == "y" and isVowel(str[1])

  return non_y_cons or y_cons

# return true if ends with normal cons or 'qu'
def last_is_cons(str: str) -> bool:
  last = str[len(str) - 1]
  lasttwo = str[len(str)-2] + last

  return isConsonant(last) or lasttwo == "qu"





In [135]:
def pick_syllable(
      cult: Culture, next: str, part_choice: str,
      culture_pool=None ) -> dict:

    if culture_pool is None:
        culture_pool = all_cultures

    syl_type: str
    idx: int

    # Validate 'next' input - must be one of the expected codes
    if next not in {'c', 'v', 'C', 'V'}:
      raise ValueError(f"bad input for 'next': {next}")

    # Remap 'next' codes to syllable type keys for sorting buckets
    # 'c' or 'C': previous ended Vowel, current should start Consonant
    # 'v' or 'V': previous ended Consonant, current should start Vowel
    if next == 'c': # Normal consonant start
      syl_type = "c_W"
    elif next == 'v': # Normal vowel start
      syl_type = "v_W"
    elif next == 'C': # Obligate consonant start (e.g., after single vowel)
      syl_type = "C_W"
    elif next == 'V': # Obligate vowel start (e.g., after single consonant)
      syl_type = "V_W"
    else:
      # This branch should ideally not be reached due to the initial validation.
      # Added for robustness, or can be removed if initial validation is deemed sufficient.
      raise ValueError(f"Unhandled 'next' value encountered: {next}")

    if part_choice == "prefix":
      word_part = cult.pre
    elif part_choice == "middle":
      word_part = cult.mid
    else: # part == "suffix"
      word_part = cult.suf


    # choose what pool to get the index from
    while word_part.get(syl_type) == []:
        cult = random.choice(culture_pool)
        if part_choice == "prefix":
          word_part = cult.pre
        elif part_choice == "middle":
          word_part = cult.mid
        else: # part == "suffix"
          word_part = cult.suf

    syls_of_type = word_part.get(syl_type)
    all = word_part.get("all")

    idx = random.choice(syls_of_type)


    # choose the syllable associated with this index
    syllable = word_part.get("all")[idx]

    # if a prefix was chosen in fallback, make the
    # first letter lowercase (hypen aware)
    non_hyph_upper: bool = syllable[0].isalpha() and syllable[0].isupper()

    if part_choice != "prefix" and non_hyph_upper:
      syllable = syllable[0].lower() + syllable[1:]

    result: dict = {
        "syl": syllable,
        "cult": cult,
        "syl_type": next}

    return result

In [136]:
import random

##################### v2
def generateName(cultures: list[Culture]) -> None:

  """############### PREFIX ###############"""
  name: str
  cult: Culture = random.choice(cultures)

  # prefix: random of all this cultures's prefixes
  syl: str = random.choice(cult.pre.get("all"))
  debug1: str = ("...")

  info1: str = (cult.name)
  info2: str = (syl)

  name = syl;   """add it to name"""


  next = getNext(syl)

  """############### MIDDLE ###############"""
  syl: str = ""
  if random.choice([True, False]):
    # choose a new culture
    cult = random.choice(cultures)

    pickSyl = pick_syllable(cult, next, "middle")

    syl = pickSyl.get("syl")

    debug1 = (debug1+ " — " + pickSyl.get("syl_type"))

    if syl != "":
      next = getNext(syl)

      info1 = (info1+ " — " + pickSyl.get("cult").name)
      info2 = (info2+ " — " + syl)

      """add it to name"""
      doubleDashes: bool = not name[len(name) - 1].isalpha() and not syl[0].isalpha

      if doubleDashes:
        name = name + str(syl[1:])
      else:
        name = name + syl

    # 25% chance of second mid syllable, same culture
    if random.choice([True, False]):
      pickSyl = pick_syllable(cult, next, "middle")

      syl = pickSyl.get("syl")

      debug1 = (debug1+ " — " + pickSyl.get("syl_type"))

      if syl != "" and len(name) + len(syl) < 8:
        next = getNext(syl)

        info1 = (info1+ " — " + pickSyl.get("cult").name)
        info2 = (info2+ " — " + syl)

        """add it to name"""
        doubleDashes: bool = not name[len(name) - 1].isalpha() and not syl[0].isalpha()

        if doubleDashes:
          name = name + str(syl[1:])
        else:
          name = name + syl


  """############### SUFFIX ###############"""
  suffix_pool = all_cultures

  if name[-1] in {"-", "'"}:
      suffix_pool = [c for c in all_cultures if c.name not in {"Roman", "Greek"}]

  # choose a new culture
  cult = random.choice(suffix_pool)

  pickSyl = pick_syllable(cult, next, "suffix", suffix_pool)
  syl = pickSyl.get("syl")

  debug1 = (debug1+ " — " + pickSyl.get("syl_type"))

  """add it to name"""
  if syl != "":

    info1 = (info1+ " — " + cult.name)
    info2 = (info2+ " — " + syl)

    """add it to name"""
    if not name[len(name) - 1].isalpha() and not syl[0].isalpha():
      name = name + str(syl[1:])
    else:
      name = name + syl



  """############### OUTPUT ###############"""
  debug_on = True

  print(name)
  print("  "+info1)
  if debug_on:
    print("  "+debug1)
  print("  "+info2 + "\n")


In [137]:
## each count as one consonant
digraphs: list[str] = ["ch", "ph", "th", "ck", "sh", "qu", "wh",
                       "kn", "kh", "kh"]

def getNext(syl: str) -> str:
  # Handle single-character syllables
  if len(syl) == 1:
    if isVowel(syl[0]):
      return 'C' # single vowel, next is obligate consonant start
    elif isConsonant(syl[0]):
      return 'V' # single consonant, next is obligate vowel start
    else:
      # Fallback for unexpected char (not vowel/consonant)
      return 'c'

  # replace a trailing digraph with a single consonant
  if syl[-2:] in digraphs:
    syl = syl[:-2] + "c"

  # For syllables with length >= 2, use last_is_cons helper function
  if last_is_cons(syl):
    # if consonant cluster end, next is obligate vowel start
    if isConsonant(syl[-2:-1])
      return 'V'
    else
      return 'v' # Ends in a consonant, next starts with a vowel
  else:
    return 'c' # Ends in a vowel, next starts with a consonant

In [138]:
#  syllables for all cultures
all = {
  #### EGYPTIAN ####
  "e_p": [
      "A","Nu", "Ha", "Dje", "Djo", "Amen", "Amon", "Anu", "Bas", "Hat",
      "Sob", "Nef", "Nefer", "Sen", "Ahm", "Akhe", "Khu", "Kh", "Sut", "Ka-",
      "Khn", "Kha", "Ram", "Thut", "Sme", "Set", "Pa-", "Aa", "Aam", "Hat-",
      "Dsj", "Dsja", "Osir", "Isi", "Ankh", "Ar", "Nub", "He"
    ],

  "e_m": [
      "ph", "phth", "ho", "huti", "nou", "shep", "kn", "-sch", "-nefer-",
      "de", "phr", "khm", "kh", "dj", "mh", "khk", "er", "utme", "-at-", "-en-",
      "hm", "khe", "nemh", "khn", "fer", "f", "pta", "djm", "fer", "-sa",
      "auten", "nkhk", "utmose", "ankh", "ankha", "kham", "-na", "khur-",
       "nefer", "khma", "khe", "ekhma", "khs", "khsep", "-anh"
    ],

  "e_s": [
      "are", "eru", "ys", "ek","esu", "et", "em", "at", "ten", "aa", "mat",
      "eb", "ut", "uti", "hotep", "ho", "tep", "are", "aten", "-hat", "i",
      "tari", "ti", "fre", "fre", "tem", "fra", "mose", "mon", "ket", "nofre",
      "kh", "det", "tep", "th", "nut", "sis", "mheb", "nkh", "-ap", "tef",
      "ankh", "ptah","met", "kmet", "djmet", "bek", "feru", "khamon", "khons"
      "sret", "sobek", "khufu", "psut", "phris", "-heb", "-i", "mhe", "-pu",
      "pet", "het", "pat", "-shen", "-hapi"
    ],

  #### ROMAN ####
  "r_p": [
      "Luc", "Val", "Mar", "Ovi", "Gai", "Hel", "Agrip", "Opp", "Mall",
      "C", "Tac", "Ta", "Ae", "Au", "Aure", "Cae", "Cl", "Fla", "Ru", "Lu",
      "Gl", "Gn", "Iu", "Ju", "Jul", "Liv", "Ma", "Marc", "St", "Men",
      "Max", "Maxi", "Por", "Prisc", "Quint", "Sci", "Sev", "Io", "Publ",
      "Silv", "Tit", "Va", "Vi", "Val", "Var", "Vib", "Cor", "Gall", "Ha",
      "P", "Tre", "Dru", "Septi", "Arca", "Can", "Lae", "Quatr", "Qu", "Qui",
      "Quint", "Ia", "Iac", "Flav", "Calv", "E", "Ter", "Sci", "Cre", "Cr",
      "Oct", "Octa", "Serv", "Octo", "Kae", "Jun", "Tut", "Tibur", "Tri",
      "Deci", "Decim", "Pae", "Sae", "Iust", "Bae", "Arc", "Arcad", "Arca",
      "Publi", "Sext", "Cocc", "Tert", "Secund", "Mae", "D", "Pela"
    ],
  "r_m": [
      "an", "in", "ian", "ae", "an", "en", "ar", "qu", "v", "fr", "aeb", "ill",
      "c", "n", "sc", "t", "p", "aia", "l", "lian", "mil", "qui", "ll", "vern",
      "ss", "ssian", "ian", "aud", "au", "ci", "m", "iab", "ael", "Nov", "nian"
      "up", "rc", "nl", "cell", "ae", "ver", "lp", "rqu", "quin", "ptim",
      "nti", "nt", "ell", "ni", "arat", "pst", "cl", "clect", "eia", "mir",
      "cret", "lla", "atin", "tor", "sent", "cil", "lian", "ian", "at", "spi",

    ],
  "r_s": [
      "ia", "ius", "us", "ina", "ella", "a", "iana", "aeus", "ars", "io",
      "ian", "ianus", "allas", "llus", "icus", "ilius", "eius", "sus", "sca",
      "ntia", "ata", "mus", "aeus", "llo", "llia", "llius", "aius", "spex",
      "eria", "erio", "cia", "tia", "tio", "cio", "cius", "ilina", "ssus"
    ],

  #### GREECE ####
  "g_p": [
      "Lys", "Kal", "Ari","Rho", "Kor", "Arte", "Ae", "Ch", "Chl",
      "Deme", "Dem", "Tel", "Tele", "Thou", "Eu", "A", "Tha", "Da",
      "Her", "Hesp", "Ny", "Ph", "Per", "Pel", "Phi", "Th", "Theo",
      "Tha", "Xa", "Xe", "Xeno", "Zo", "Ze", "Louk",
    ],
  "g_m": [
      "er", "in", "th", "t", "kr", "d", "tr", "k", "ll",
      "ph", "gor", "ch", "nt", "sch", "l", "ga", "pe", "ng", "ei",
      "el", "gath", "x", "xan", "lk", "ia", "mp", "el", "rk",
      "ari", "sth", "ian", "nth", "pho", "phe" "sthe", "hem", "rg"
      "thy", "pp", "mach", "ymph", "mph", "meni", "ych", "ou",
    ],
  "g_s": [
      "ius", "ias", "is", "as", "ache", "one", "is", "des", "os",
      "ios", "us", "es", "eas", "eus", "ate", "ates", "ice", "sos",
      "os", "oë", "eus", "sia", "kles", "ion", "e", "ton", "eon",
      "phon", "es", "d", "od", "iod", "ene", "ates", "kles", "che",
    ],

  #### NEAR EAST ####
  "ne_p": [
      "Ha", "Hamm", "Has", "Nab", "Nabo", "Nabu", "Nebu", "Esar", "E",
      "Bel", "Sar", "Sha", "Shal", "En", "Enh", "Me", "Tia", "Mesi",
      "Sin-", "A", "Lam", "Asor", "Ashur", "Lars", "Ea-", "Geme", "Iaa",
      "Ari", "I", "Baa", "Ish", "Isht", "Ensh", "Ara", "Uba", "Gem",
      "Ishm", "Ipqu-", "Ar'", "Aha", "Ku-", "Ga", "Ni", "Nii", "Enhe"
    ],

  "ne_m": [
      "mmu", "dru", "apla", "pol", "mmur", "nn", "zz", "mil", "nezz", "'ii",
      "sharra", "sharr", "sha", "sh", "kk", "ira", "ban", "shur", "ch", "mki",
      "dn", "adn", "lm", "man", "a", "shl", "nhe", "dd", "rh", "shm", "u", "i",
      "hati-", "-ban", "shkir", "di-", "-Sharr", "kha", "-na", "zi", "pqu-",
      "shtu", "z", "methr", "ii", "-Gam", "e", "ata", "dugg", "unz", "shtu",
      "quu", "gsumm", "urza", "hun", "-Wa", "qr", "shti-", "shkiri", "isht",
      "bni-", "ww", "'siu", "uiq", "qa", "iiarqu", "rqus", "zur-", "-Ka",
      "ssunu", "-Aya", "kiri", "kiti", "k"
    ],

  "ne_s": [
      "ssar", "assar", "usur", "nna", "car", "ar", "zzar", "rdoch", "it",
      "don", "rri", "i", "nezzar", "ton", "gon", "mis", "amis", "lshu", "shu",
      "bi", "neser", "mat", "at", "ltum", "tum", "rra", "shua",
      "itum", "na", "-ea", "-idi", "-eil", "-dan", "shi", "-nasir", "paa", "esi",
      "eesu", "-Aya", "kusu", "bitu", "lat", "aat", "ggat", "aa", "unu", "annu",
      "ka", "uu", "kir", "lnu", "qrat", "llat", "qqa", "rqusu", "qusu", "uun",
      "ti", "duana", "ana", "uana", "iti", "kaa",
    ]
}


# create class instance for each culture
Egypt = Culture(all.get("e_p"), all.get("e_m"), all.get("e_s"), "Egyptian")
Rome = Culture(all.get("r_p"), all.get("r_m"), all.get("r_s"), "Roman")
Greece = Culture(all.get("g_p"), all.get("g_m"), all.get("g_s"), "Greek")
Near_East = Culture(all.get("ne_p"), all.get("ne_m"), all.get("ne_s"), "Near East")

# list of all cultures/classes
all_cultures = [Egypt, Rome, Greece, Near_East]


# Main
AFTER pressing "Run All", press the play button below.

In [150]:
# <- Hover over me!! Click that button!!

for i in range(5):
  generateName(all_cultures)

Quatr-ap
  Roman — Egyptian
  ... — v
  Quatr — -ap

Tert-napet
  Roman — Egyptian — Egyptian
  ... — v — c
  Tert — -na — pet

Kalar
  Greek — Near East
  ... — v
  Kal — ar

Larsa
  Near East — Roman
  ... — v
  Lars — a

Aninar
  Greek — Roman — Roman — Near East
  ... — C — V — v
  A — n — in — ar



# hide
don't worry about this

In [140]:
debug: bool = False
if debug:
  cult = Rome

  print(cult.name)

  pickSyl = pick_syllable(cult, 'v', "suffix")

  print("  "+ pickSyl.get("syl") + " - " + pickSyl.get("cult").name)

In [141]:
debug: bool = True

def print_test(cult, word_part, syl_type):

  syls_of_type = word_part.get(syl_type)
  all = word_part.get("all")

  print(cult.name+" - "+str(len(syls_of_type))+"")
  # print("  "+str(all))

  result: str= ""
  for i, idx in enumerate(syls_of_type):
    result = result + ", "+ all[idx]

  result = result.removeprefix(", ")
  result = "  [" + result + "]"
  print(result)
  print()

if debug:
  for cult in all_cultures:
    word_part = cult.suf
    syl_type = "c_W"
    print_test(cult, word_part, syl_type)


Egyptian - 47
  [ten, mat, hotep, ho, tep, -hat, tari, ti, fre, fre, tem, fra, mose, mon, ket, nofre, kh, det, tep, th, nut, sis, mheb, nkh, -ap, tef, ptah, met, kmet, djmet, bek, feru, khamon, khonssret, sobek, khufu, psut, phris, -heb, -i, mhe, -pu, pet, het, pat, -shen, -hapi]

Roman - 0
  []

Greek - 1
  [d]

Near East - 44
  [ssar, nna, car, zzar, rdoch, don, rri, nezzar, ton, gon, mis, lshu, shu, bi, neser, mat, ltum, tum, rra, shua, na, -ea, -idi, -eil, -dan, shi, -nasir, paa, -Aya, kusu, bitu, lat, ggat, ka, kir, lnu, qrat, llat, qqa, rqusu, qusu, ti, duana, kaa]



In [142]:
# testing is consonant function
#########################
debug_on = False
if debug_on:
  test = [
        "ph", "phth", "ho", "huti", "nou", "shep", "kn", "-sch", "-nefer-",
        "de", "phr", "khm", "kh", "dj", "mh", "khk", "er", "utme", "-at-", "-en-",
        "hm", "khe", "nemh", "Ae", "Au", "Aure", "Cae", "Cl", "Fla", "Ru", "Lu",
        "Gl", "Gn", "Iu", "Ju", "Jul", "Liv", "Ma", "Marc", "St", "Men",
        "Max", "Maxi", "Por", "Prisc", "Quint", "Sci", "Sev", "Io", "Publ",
        "Silv", "Tit", "Va", "Vi", "Val", "Var", "khsep", "-anh"
      ]
  L1 = ""
  L2 = ""
  for str in test:
    if all_consonants(str):
      L1 = L1 + ", " + str
    else:
      L2 = L2 + ", " + str

  L1 = L1.removeprefix(", ")
  L2 = L2.removeprefix(", ")

  print("all consonants: [" + L1 + "]")
  print("not all consonants: [" + L2 + "]")